## データの概要



*   データ収集期間 : 2021年11月から12月
*   調査対象 : 大学やその他団体が主催しているミスター・ミスコンテストに出場している人のInstagramアカウント（300前後のアカウントを対象として調査）
*   ユーザー情報にアクセスした際, 正常に情報が取得できた150のアカウントのデータを解析



## 準備

### アクセストークンの取得

一つ目のアクセストークン（英数字）を
<br>
https://developers.facebook.com/tools/explorer/
<br>
から取得。

アプリID（数字）とapp secret（英数字）を準備し、一個目のアクセストークンを含め、これらを含めたURLをブラウザで検索。

以下のURLを打ち込みブラウザに打ち込めば良い。


https://graph.facebook.com/v4.0/oauth/access_token?grant_type=fb_exchange_token&client_id={ここにアプリIDを入力。中括弧は不要}&client_secret={ここにapp secretを入力。中括弧は不要}&fb_exchange_token={ここに一つ目のアクセストークンを入力。中括弧は不要}

すると二つ目のアクセストークン（英数字）が得られる。




上の結果（二個目のアクセストークン）を使って以下URLに飛ぶ。

https://graph.facebook.com/v4.0/me?access_token={ここに二個目のアクセストークンを入力。中括弧は不要}

この画面に表示されているidを使って三個目のアクセストークンを取得する。僕の場合は以下（ダミー）。

{
   "name": "XXX",
   "id": "123456789012345"
}

以上の結果を用いて以下URLにアクセス（123456789012345は上のid）。

https://graph.facebook.com/v4.0/123456789012345/accounts?access_token={ここに二個目のアクセストークンを入力。中括弧は不要}

すると三個目のアクセストークンを取得できる。

これを用いてInstagramビジネスIDを取得する。

アクセストークンを三個目のものに書き換えて、me以下のパラメータも書き換えた後、その右の送信ボタンを押すことでInstagramビジネスIDを取得できた。僕の場合は以下（ダミー）。

InstagramビジネスID　: 01234567890123456

https://developers.facebook.com/docs/graph-api?locale=ja_JP
でfacebook graph APIの最新バージョンを確認した。

ユーザーネームは@以下の文字列のこと。

コーディングの参考文献は以下。
> [Instagram Graph APIとPythonでInstagramの「ユーザー情報、メディア情報」を取得する](https://gaaaon.jp/blog/instagram)
> [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)　<br>
> [マッピング型 --- dict](https://docs.python.org/ja/3/library/stdtypes.html?#mapping-types-dict) <br>
> [requests 2.26.0](https://pypi.org/project/requests/)


In [ ]:
import pandas as pd
import requests

business_account_id = '01234567890123456'  # InstagramビジネスID
token = 'ABCdef123'  # 3個目のアクセストークン
fields = 'name,username,biography,follows_count,followers_count,media_count'


def get_user_info(usernames):
  col = ['name', 'username', 'id', 'followers_count', 'biography', 'follows_count', 'media_count']
  d = {c:[] for c in col}

  for username in usernames:
    request_url = "https://graph.facebook.com/v12.0/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
    r = requests.get(request_url)
    if 'business_discovery' in r.json():
      for c in col:
        d[c].append(r.json()['business_discovery'][c])
      df = pd.DataFrame(data=d)
  return df


usernames = ['chen_.x_y._', 'rui_hibi', 'yunosu__1024', 'tatsuya._nami', 'mr_keioygm21_5', 'ms_keioygm21_1', 'ms_keioygm21_2', 'risa.shindo.620', 'sakura_aono_', 'ms_keioygm21_5']


2回目以降は過去に作ったcsvファイルをここから読み込み。

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
data_mrms = pd.read_csv('/content/drive/My Drive/data_mrms.csv').drop(['Unnamed: 0'], axis=1)
data_mrms

Mounted at /content/drive


,name,username,followers_count,biography,follows_count,media_count,sex,reel,movie_not_reel,reel_and_movie
0,鄭 幸陽 chenxingyang,chen_.x_y._,1455,◻️ミスター慶應理工コンテスト2021\n◻️慶應\n◻️理工学部応用化学科,1637,73,1.0,1.0,0.0,0.0
1,日比 琉偉,rui_hibi,1990,慶應義塾大学理工学部\nミスター慶應理工コンテスト準GP🥈\n用件のある方はDMまで🙇‍♂️,1612,63,1.0,1.0,1.0,1.0
2,伊藤 優之介,yunosu__1024,950,応援ありがとうございました📣,868,29,1.0,0.0,0.0,0.0
3,濤川 達也 |Namikawa Tatsuya,tatsuya._nami,2091,◽︎慶應義塾大学2年\n◽︎ミスター慶應理工2021\n◽︎湘南🌊/177cm\n◽︎お仕事...,1682,32,1.0,1.0,0.0,0.0
4,佐藤 ききよ,mr_keioygm21_5,2356,▪️ミスター慶應理工2021 GP👑\n▪️NEXT>>> MR OF MR CAMPUS ...,6728,40,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
136,ミクス日芸No.3 MARIA ‐ ﾏ ﾘ ｱ ‐,___jazmin.arcoiris,647,Mx日芸コンテスト2021🌟💎 DHC賞🏅\nNon-Binary ; they / the...,533,44,0.0,NaN,NaN,NaN
137,Rio,urlocalmixedgirll,722,NUA Film \nミクス日芸コンテスト2021グランプリ👑,865,31,0.0,NaN,NaN,NaN
138,安田令,misstmdu202101,800,一般的歯学生\nDorrydoll賞、準グランプリありがとうございました💐\n被写体依頼、そ...,270,59,0.0,NaN,NaN,NaN
139,ミス熊大No.5 安川未蘭,miss_kumadai_no.5,1920,第十回ミス熊大コンテストグランプリ👑🤍\n応援本当にありがとうございました😭✨,1449,5,0.0,NaN,NaN,NaN


各投稿の情報を取れる範囲でとってくる。
> [IGメディア](https://developers.facebook.com/docs/instagram-api/reference/ig-media)<br>
> [IGユーザービジネスディスカバリー](https://developers.facebook.com/docs/instagram-api/reference/ig-user/business_discovery)

25件までしか投稿がとれないのでそれ以降の投稿をとる処理を追加。

以下のサイトを参考にした（公式では`media.after`の話が見つけられなかった）。

> [Instagram Graph APIとPythonでInstagramの「ユーザー情報、メディア情報」を取得する](https://www.cloudnotes.tech/entry/instagram_graph_api)
<br>
> [pandas.Series.value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)

In [ ]:
import pandas as pd
import requests

business_account_id = '01234567890123456'  # InstagramビジネスID
token = 'ABCdef123'  # 3個目のアクセストークン
#fields = 'name,username,followers_count,media_count,media{caption,comments_count,like_count,media_type,media_product_type,timestamp}'
fields = 'name,username,followers_count,media_count,media{caption,comments_count,media_type,media_product_type,timestamp}'
media_fields = 'caption,comments_count,media_type,media_product_type,timestamp'

col = ['name', 'username', 'followers_count']
col_media = ['caption', 'comments_count', 'media_type', 'media_product_type', 'timestamp']
d = {c:[] for c in col + col_media}

# 25件以上の場合の処理も追加
def get_user_media_info(username):
  request_url = "https://graph.facebook.com/v12.0/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
  r = requests.get(request_url)
  d['name'].append(r.json()['business_discovery']['name'])
  d['username'].append(r.json()['business_discovery']['username'])
  d['followers_count'].append(r.json()['business_discovery']['followers_count'])
  #N = r.json()['business_discovery']['media_count']
  n = len(r.json()['business_discovery']['media']['data'])
  for i in range(n):
    for c in col_media:
      d[c].append(r.json()['business_discovery']['media']['data'][i][c])
  #print(len(d['caption']), d['caption'])
  if 'after' in r.json()['business_discovery']['media']['paging']['cursors']:
    next_token = r.json()['business_discovery']['media']['paging']['cursors']['after']
    #print(next_token)
    while next_token != None:
      request_url = "https://graph.facebook.com/v12.0/{business_account_id}?fields=business_discovery.username({username}){{media.after({next_token}){{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token,next_token=next_token)
      r = requests.get(request_url)
      #print(r.json()['business_discovery']['media']['data'])
      n = len(r.json()['business_discovery']['media']['data'])
      
      for i in range(n):
        for c in col_media:
          d[c].append(r.json()['business_discovery']['media']['data'][i][c])
      if 'after' in r.json()['business_discovery']['media']['paging']['cursors']:
        next_token = r.json()['business_discovery']['media']['paging']['cursors']['after']
      else:
        next_token = None
  
  print(d)
  print(d['name'][0])
  N = len(d['caption'])
  d['name'] = [d['name'][0]] * N
  d['username'] = [d['username'][0]] * N 
  d['followers_count'] = [d['followers_count'][0]] * N
  df = pd.DataFrame(data=d)
  #df['name'] = r.json()['business_discovery']['name']
  print(df['media_type'].value_counts())
  print(df['media_product_type'].value_counts())
  return df, df['media_type'].value_counts()
  
  
  #for c in col:
  #  d[c].append(r.json()['business_discovery'][c])
  #

usernames = ['chen_.x_y._', 'rui_hibi', 'yunosu__1024', 'tatsuya._nami', 'mr_keioygm21_5', 'ms_keioygm21_1', 'ms_keioygm21_2', 'risa.shindo.620', 'sakura_aono_', 'ms_keioygm21_5']
df_tmp, tmp = get_user_media_info('chen_.x_y._')
df_tmp

{'name': ['鄭 幸陽  chenxingyang'], 'username': ['chen_.x_y._'], 'followers_count': [1454], 'caption': ['先日出演させて頂きましたCanCam40th BirthdayNight の様子です！\n\nほんとに楽しかった😊😊\n\nぜひアーカイブがあがっているので見逃した方はYouTubeで見てください！！\n\n #CanCam40 #cancam \n\nhttps://www.youtube.com/watch?v=8yoOdsz1h_o', '2021.11.28\n\n #養老渓谷', 'めちゃくちゃに美味しいしこれで2000円っていうコスパ😚😚\n\nなかなか人形町行かないけどリピートしたい、、\n #人形町グルメ', '秋のイチョウ並木\n服は冬\n・\n明日19時からCanCam40th Birthday Nightに出演します。\nぜひ見てください🙇\u200d♂️', '美味しいし\n何気に初恵比寿。\n #恵比寿グルメ  #オムライス #恵比寿ガーデンプレイス', '慶應理工ミスターコンテスト2021の協賛品としてaqua savon の香水を頂きました！\n\n #アクアシャボンスポーツ #aquasavonsports  #オードトワレ #香水 #フレグランス', '*\n3ヶ月にも渡ったミスターコンテストが幕を閉じ、自分はミスターメンズリゼ賞を受賞致しました。\n改めてこんなに素敵な賞をいただけたのは普段から支えてくれた身近な友人達、同じ学科で共に頑張れたミスNo.3のりさ、投票報告欠かさずしてくれた皆さん、ミクチャ配信を盛り上げてくれた人達、慶應理工ミスコンの運営そして共にエントリーできたみんな、思い出すだけでキリがないほどのたくさんの人のおかげだと感じています。\n*\nここ数日、他大のミスコンの結果を見届け、1ファンとして一喜一憂して、ふと自分も同じように見てくれた人達の心を何かしらの形で動かせたのかなと思いました。\n*\nもし誰かの心を少しでも動かせるような活動ができていたら嬉しいななんて思いながら、これからも誰かのために頑張れる人になれるよう努力したいと感じました。\n*\n改めまして皆さんのおかげで素敵な3ヶ月を過ごすことが出来ま

,name,username,followers_count,caption,comments_count,media_type,media_product_type,timestamp
0,鄭 幸陽 chenxingyang,chen_.x_y._,1454,先日出演させて頂きましたCanCam40th BirthdayNight の様子です！\n\...,2,CAROUSEL_ALBUM,FEED,2021-12-06T13:25:36+0000
1,鄭 幸陽 chenxingyang,chen_.x_y._,1454,2021.11.28\n\n #養老渓谷,2,CAROUSEL_ALBUM,FEED,2021-11-29T11:56:15+0000
2,鄭 幸陽 chenxingyang,chen_.x_y._,1454,めちゃくちゃに美味しいしこれで2000円っていうコスパ😚😚\n\nなかなか人形町行かないけど...,2,CAROUSEL_ALBUM,FEED,2021-11-26T02:02:21+0000
3,鄭 幸陽 chenxingyang,chen_.x_y._,1454,秋のイチョウ並木\n服は冬\n・\n明日19時からCanCam40th Birthday N...,0,CAROUSEL_ALBUM,FEED,2021-11-24T04:47:36+0000
4,鄭 幸陽 chenxingyang,chen_.x_y._,1454,美味しいし\n何気に初恵比寿。\n #恵比寿グルメ #オムライス #恵比寿ガーデンプレイス,1,CAROUSEL_ALBUM,FEED,2021-11-21T10:14:42+0000
...,...,...,...,...,...,...,...,...
66,鄭 幸陽 chenxingyang,chen_.x_y._,1454,みなさんこんばんは！！今日も1日お疲れ様です！\n\n今日は三田ランチの時の写真をあげます📸...,11,CAROUSEL_ALBUM,FEED,2021-08-05T11:52:54+0000
67,鄭 幸陽 chenxingyang,chen_.x_y._,1454,みなさんこんばんは！🌃\n\n写真は昨日と同じ日のものです！僕は今、卓球男子団体準決勝を見て...,4,CAROUSEL_ALBUM,FEED,2021-08-04T11:20:26+0000
68,鄭 幸陽 chenxingyang,chen_.x_y._,1454,みなさんこんばんは！\n\n僕は今日、ワクチンの副作用で1日ダウンしておりました😰\n\nそ...,5,CAROUSEL_ALBUM,FEED,2021-08-03T10:35:16+0000
69,鄭 幸陽 chenxingyang,chen_.x_y._,1454,こんばんは！今日も1日お疲れ様でした😌暑くて仕方なかったのですが聖火台を見にお台場まで行きま...,6,CAROUSEL_ALBUM,FEED,2021-08-01T11:40:07+0000


csv出力をする。

> [pandas.DataFrame.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
#df_keio.to_csv('/content/drive/My Drive/data_keioyagami.csv')
df_keio_tmp.to_csv('/content/drive/My Drive/data_keioyagami2.csv')

Mounted at /content/drive


他大学についても調べていく。とりあえず100件くらい。

> [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)

In [ ]:
usernames_doshisha = ['mcd2021_tm01', 'mcd2021_yr02', 'mcd2021_or03', 'mcd2021_fr05', 'mcd2021_ta06']
df_doshisha = get_user_info(usernames_doshisha)
df_doshisha

,name,username,id,followers_count,biography,follows_count,media_count
0,友廣 南実《ミス同志社 No.1》,mcd2021_tm01,17841448658566104,22584,友廣 南実(ともひろ みなみ) 2年生\n・Miss Doshisha 2021 GP✨\n...,2013,168
1,山口 莉央《ミス同志社 No.2》,mcd2021_yr02,17841448633847213,3553,"""2002.06.06（19）155cm""\n*ミスキャンパス同志社2021🍓🧸\n*第70...",958,151
2,大野 凜《ミス同志社 No.3》,mcd2021_or03,17841448123446351,11610,▫️ミスキャンパス同志社2021\n▫️第‬68回今宮戎神社福娘‪🎋\nたくさんの応援ありが...,2492,138
3,藤中 里緒 Rio Fujinaka,mcd2021_fr05,17841448342078508,12956,▪︎Miss Campus Doshisha2021 準GP\n▪︎第70回今宮戎神社福娘\...,2191,152
4,田邉 彩香/ Ayaka Tanabe,mcd2021_ta06,17841448244313022,5989,2001.10.29 (20) 164cm\nMiss Campus Doshisha 20...,928,136


In [ ]:
df = pd.concat([df_keio_tmp, df_doshisha], ignore_index=True)
df

,name,username,id,followers_count,biography,follows_count,media_count
0,鄭 幸陽 chenxingyang,chen_.x_y._,17841448937600111,1455,◻️ミスター慶應理工コンテスト2021\n◻️慶應\n◻️理工学部応用化学科,1637,73
1,日比 琉偉,rui_hibi,17841449004527753,1990,慶應義塾大学理工学部\nミスター慶應理工コンテスト準GP🥈\n用件のある方はDMまで🙇‍♂️,1612,63
2,伊藤 優之介,yunosu__1024,17841449088525534,950,応援ありがとうございました📣,868,29
3,濤川 達也 |Namikawa Tatsuya,tatsuya._nami,17841448730908064,2091,◽︎慶應義塾大学2年\n◽︎ミスター慶應理工2021\n◽︎湘南🌊/177cm\n◽︎お仕事...,1682,32
4,佐藤 ききよ,mr_keioygm21_5,17841449022017346,2356,▪️ミスター慶應理工2021 GP👑\n▪️NEXT>>> MR OF MR CAMPUS ...,6728,40
5,齋藤萌花,ms_keioygm21_1,17841448979479106,3204,ミス慶應理工コンテスト2021\nファイナリスト,1687,78
6,上手 理花 / Rika Kamite,ms_keioygm21_2,17841449121524353,2281,▫️慶応義塾大学3年\n▫️Miss Keio Ygm 2021\n▫️ハウスコム Web ...,1039,118
7,新藤 利佐 Risa Shindo,risa.shindo.620,17841448550193556,9278,◾︎Miss Keio Ygm 2021\n◾︎2000/6/20\n◾︎168 cm,1067,149
8,青野 さくら,sakura_aono_,17841448669559535,4121,ミス慶應理工2021GP\nたくさんの応援ありがとうございました✨,1661,74
9,江口 玲恵 ミス慶應理工 No.5,ms_keioygm21_5,17841448754306862,1617,ミスミスター慶應理工コンテスト2021\nEntry No.5 \n理工学部1年 学門C\n...,748,36


In [ ]:
# 関大12月5日まで
usernames_kandai = ['mck21_mr01', 'mck_21mr02', 'mck21_mr04', 'mck21_mr05', 'mck21_mr06', 'mck21_miss01', 'mck21_miss02', 'mck21_miss03', 'mck21_miss04', 'mck21_miss05', 'mck21_miss06']
df_kandai = get_user_info(usernames_kandai)
df = pd.concat([df, df_kandai], ignore_index=True)
df_kandai

,name,username,id,followers_count,biography,follows_count,media_count
0,大西悠雅 ミスター関大No.1,mck21_mr01,17841448561913608,1424,◾︎ Mr Kandai 2021 No.1\n◾︎ 2000/11/08 180cm\n...,996,42
1,福田敦也,mck_21mr02,17841448759790453,1995,2000/Hyogo→Osaka\n【福田敦也の挑戦物語】\n・Mr &Miss Campa...,1944,122
2,内野真之介 ミスター関大No.4,mck21_mr04,17841448816307834,583,普段はぼっちでYouTubeをしてます📸\nYouTube【ぼっち大学生のすけ】で検索💫\n...,556,23
3,松村竜明【たつ】,mck21_mr05,17841448448642059,2933,Mr&Miss Campus KANDAI 2021 GP🥇\n・YouTube【メラビアン...,2435,61
4,木村太一 ミスター関大 No.6,mck21_mr06,17841448358855436,3514,大阪の98単位落として留年した5回生\nバイクと音楽を愛しております🏍🎸\n🏍TRIUMPH...,3750,53
5,北田理沙 ミス関大No.1,mck21_miss01,17841448397163803,4672,▶︎2001/12/26 商学部2回生\nお仕事依頼は公式SNSまで💚💐→@mc_kanda...,1258,84
6,市川舞裕 Mahiro Ichikawa,mck21_miss02,17841448542867342,2208,◾︎ 𝐌𝐢𝐬𝐬 𝐜𝐚𝐦𝐩𝐮𝐬 𝐊𝐀𝐍𝐃𝐀𝐈 𝟐𝟎𝟐𝟏\n◾︎ 𝐨𝐬𝐚𝐤𝐚,1612,39
7,岩瀬 栞采,mck21_miss03,17841448763358642,3211,2001.10.20 @p_o_o_h____1020 \n🌸関大ミスキャン2021 準G...,1097,93
8,天野 こころ ˙θ˙,mck21_miss04,17841448391674194,3527,151㌢ / 22 / kobe / 🐥\nHandmade Accessory comin...,3412,180
9,倉田ひな ミス関大No.5,mck21_miss05,17841448515657822,2927,𝘔𝘳 &𝘔𝘪𝘴𝘴 𝘊𝘢𝘮𝘱𝘶𝘴 -𝘒𝘈𝘕𝘋𝘈𝘐- 2021 \n＊DMの対応は致しかねます🐈🌿,1338,157


In [ ]:
# 関学11月下旬まで
# ミスターコンなさそう
usernames_kangaku = ['mckg2021_no1', 'mckg2021_no2', 'mckg2021_no3', 'mckg2021_no4', 'mckg2021_no5', 'mckg2021_no6']
df_kangaku = get_user_info(usernames_kangaku)
df = pd.concat([df, df_kangaku], ignore_index=True)
df_kangaku

,name,username,id,followers_count,biography,follows_count,media_count
0,奥田 歩美 / Ami Okuda,mckg2021_no1,17841448281770846,7083,𝟤𝟢𝟢𝟢.𝟣𝟤.𝟤𝟨(𝟤𝟢) 𝘖𝘴𝘢𝘬𝘢\n◽️ミス関学 𝟤𝟢𝟤𝟣 \n◽️『𝖱𝖺𝗒』専属読...,2963,204
1,ミス関学 生田 彩 (𝕀𝕜𝕦𝕥𝕒 𝔸𝕪𝕒),mckg2021_no2,17841448333009334,5975,◾︎𝗠𝗶𝘀𝘀 𝗖𝗮𝗺𝗽𝘂𝘀 𝗞𝘄𝗮𝗻𝘀𝗲𝗶 𝗚𝗮𝗸𝘂𝗶𝗻 𝟮𝟬𝟮𝟭 👗🌙\n◾︎ 𝗮𝗴𝗲𝟮𝟭...,3151,186
2,小西 美帆,mckg2021_no3,17841448026959443,1808,"🎨デジタルクリエイター(Digital artist, designer, illustra...",14,37
3,ミス関学No.4 神田 妃菜,mckg2021_no4,17841448415955591,10264,✨ミスキャンパス関西学院 2021 GP\n🎋第68代今宮戎神社福娘\nたくさんの応援を本当...,2840,173
4,田中 風花/ FUKA TANAKA,mckg2021_no5,17841448834120442,2516,☆ミス関学2021\n☆被写体、サロンモデルetc...\n🌷お仕事、ご依頼はDMにて🌷,2120,106
5,山﨑 里咲,mckg2021_no6,17841448039228984,10025,Yamasaki Risa\nミスキャンパス関西学院2021 準GP\n沢山の応援ありがとう...,2321,167


In [ ]:
# 11月上旬千葉大
usernames_chiba = ['mrhrtooooo', 'chibamr2021_no3', 'kokii_chocobii', 'tsubasa_hosogai', 'chibams2021_no1', 'chibams2021_no2', 'chibams2021_no3']
df_chiba = get_user_info(usernames_chiba)
df = pd.concat([df, df_chiba], ignore_index=True)
df_chiba

,name,username,id,followers_count,biography,follows_count,media_count
0,森洸人,mrhrtooooo,17841448519647266,811,▪︎千葉大学 Mr.Contest 2021 準GP\n▪︎muscle / walking...,158,52
1,ミスター千葉大 No.3 是久 紫郎(これひさ しろう),chibamr2021_no3,17841448646542598,2388,応援ありがとうございました😊🙏‼️\n相互フォローやめました\n▫️園芸学部4年🌱\n▫️1...,3664,100
2,しぶや こうき,kokii_chocobii,17841448723910556,947,ミスター千葉大No.4🤡\nしぶやこうきです🐕\nコムドット・吉沢亮・ベイマックス,651,113
3,細貝翼,tsubasa_hosogai,17841448390382861,1111,ミスター千葉大2021 ファイナリスト,1122,81
4,坂本桜 (Sakura Sakamoto),chibams2021_no1,17841448936572233,2744,•千葉大ミスコン2021準GP💍\nたくさんの応援ありがとうございました✨\n・2000 /...,562,56
5,窪田 望佑 (みゆ),chibams2021_no2,17841448415401885,1437,千葉大学ミスコン2021 ファイナリスト\n教育学部 音楽科 1年\n応援ありがとうございま...,931,50
6,星野愛 Ai Hoshino,chibams2021_no3,17841448851492796,3128,千葉大ミスコン2021GP💍\n国際教養学部 1年\n2003.1.12(18)新潟出身\n...,620,85


In [ ]:
# 11月上旬 筑波大コンテスト
usernames_tsukuba = ['tsukuba21__mr3']
df_tsukuba = get_user_info(usernames_tsukuba)
df = pd.concat([df, df_tsukuba], ignore_index=True)
df_tsukuba

,name,username,id,followers_count,biography,follows_count,media_count
0,ミスター筑波No.3 いとゆー（伊藤悠椰）,tsukuba21__mr3,17841448506318453,1101,▪️ TSUKUBA COLLECTION 2021 No.3\n▪️ coins17→ri...,1377,33


In [ ]:
# 11月上旬 学習院大学
usernames_gakushu = ['mr_gu21no4', 'mr_gu21no5']
df_gakushu = get_user_info(usernames_gakushu)
df = pd.concat([data_mrms, df_gakushu], ignore_index=True)
df_gakushu

,name,username,id,followers_count,biography,follows_count,media_count
0,ミスター学習院2021 No.4 井上竜之介,mr_gu21no4,17841448000983022,1245,・ミスター学習院No.4 井上竜之介（いのうえりゅうのすけ）\n・法学部法学科2年/2001...,948,43
1,倉持竣 (くらちゃん),mr_gu21no5,17841448437977568,2211,字を練習する人\n普段はハクマインというグループでYouTuberしてます。ミスター学習院フ...,175,31


In [ ]:
# ミスターオブミスターとか
usernames_mrofmr = ['mrynu2021_01', 'mrynu2021_02', 'mrynu2021_03', 'mrynu2021_04', 'mrynu2021_05', 'mrmisaki2101', 'mr_titech_2021_no.1', 'nucc21_mr01', 'nucc21_mr02', 'nucc21_mr03', 'agu21_mr1', 'agu21_mr2', 'agu21_mr3', 'agu21_mr4', 'agu21_mr5']
df_mrofmr = get_user_info(usernames_mrofmr)
df_mrofmr['sex'] = 1
df = pd.concat([data_mrms, df_mrofmr], ignore_index=True)
df_mrofmr

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,遠藤響生 ミスター横国 2021 No.1,mrynu2021_01,17841448624127709,2039,🦭MR OF MR CAMPUS CONTEST 2022🦭\nMr. YNU contes...,1739,106,1
1,津田恵佑,mrynu2021_02,17841448430725510,1069,ミスター横国2021\n2000.4.19 / 三重→横浜/大学3年生,736,50,1
2,宮島大地 ミスター横国 2021 No.3,mrynu2021_03,17841448670776564,2247,Mr.YNU Contest 2021 準GP 🦖\n2000.02.17(21),2123,117,1
3,オバタカント ミスター横国2021 No.4🌵,mrynu2021_04,17841448159595681,2671,唯我独尊、傍若無人、悪くいうとわがまま、自分勝手です\n.\n.\n↓ミスターコン本番が見れ...,1175,69,1
4,川端 昴,mrynu2021_05,17841448343849274,1361,▪️Mr.YNU2021 \n▪️1999.1.29\n※ミスコン終わったのでこれからはあり...,1162,123,1
5,りゅーーーちゃん,mrmisaki2101,17841449049529118,502,「Mr.&Miss CONTEST 2021 in MISAKI」\nエントリーNO.1\n...,345,7,1
6,石川新 ミスター東工大No.1,mr_titech_2021_no.1,17841409054860561,877,MR OF MR CAMPUS COLLECTION 2022\nミスター東工大コンテスト2...,917,48,1
7,藤田康佑 / Kousuke Fujita,nucc21_mr01,17841448682269200,2112,ミスター日大商学2021 準GP\n𝐌𝐑 𝐎𝐅 𝐌𝐑 𝐂𝐀𝐌𝐏𝐔𝐒 𝐂𝐎𝐍𝐓𝐄𝐒𝐓 𝟐𝟎𝟐𝟐...,1034,26,1
8,池澤拓真,nucc21_mr02,17841448324623403,1263,日本大学商学部 ミスターコンテスト2021 GP 👑 \ncontact→DM,1098,42,1
9,長野武蔵 日本大学商学部ミスターNo.03,nucc21_mr03,17841448656260811,295,Miss Mr.contest 2021 ファイナリスト\nアカウント移行→@gugutt_,0,1,1


In [ ]:
# ミスオブミスとか
usernames_msofms = ['senshu21_ms1', 'senshu21_ms2', 'senshu21_ms3', 'senshu21_ms4', 'senshu21_ms5', 'cankan_1225', '7namin_0', 'asiamiss2021_03', 'ring____yui', '_erin.__', 'rf99_miss05', 'rf99_miss06', 'tusambassador_2', 'tusambassador_3', 'tusambassador_5', 'toyo.dress21_02', 'toyo.dress21_03', 'toyo.dress21_04']
df_msofms = get_user_info(usernames_msofms)
df_msofms['sex'] = 0
df = pd.concat([data_mrms, df_msofms], ignore_index=True)
df_msofms

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,辻野 真衣 Mai Tsujino,tusambassador_2,17841448397093667,20661,▫︎ TUS(東京理科大)アンバサダーコンテスト2021 GP\n▫︎ Ray @ray.m...,813,80,0
1,Sayaka Miyazawa,tusambassador_3,17841448740280722,3000,TUS(東京理科大学) Ambassador Contest 2021 No.3\n・スマ...,686,69,0
2,渡辺 そら / Sora Watanabe,tusambassador_5,17841448631237106,21436,TUS ambassador contest \nお仕事のご依頼はDMにお願い致します。,1028,15,0
3,東洋大学ドレスNo.2 三浦穂乃花,toyo.dress21_02,17841448383415730,2165,東洋キャンパスコンテスト2021応援ありがとうございました♥👼🏻\nスタバ☕️レセプション💁...,2226,73,0
4,東洋大学 ドレス No.3 アハマドメヘラール,toyo.dress21_03,17841448724142815,3089,🗼Tokyo\n👽99.7.5\n💌contact☞DM,1818,159,0
5,東洋大学 ドレスNo.4 織作綾乃,toyo.dress21_04,17841448461351940,2656,Toyo Campus Contest 2021 GP\n・Tokyo\nお仕事・撮影依頼は...,1532,143,0


In [ ]:
# ミス横国
# ミスターは上のミスターオブミスターでやっている
usernames_ynu = ['msynu2021_01', 'msynu2021_04']
df_ynu = get_user_info(usernames_ynu)
df_ynu['sex'] = 0
df = pd.concat([data_mrms, df_ynu], ignore_index=True)
df_ynu

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,ミス横国 No.1 松本華,msynu2021_01,17841448499541414,11582,Ms. YNU Contest 2021 準グランプリ💍\n教育学部 教育基礎専攻 3年\n...,389,147,0
1,田中晴菜 / Haruna Tanaka,msynu2021_04,17841448686134453,2049,Ms. YNU Contest 2021\n2000/01/15\nDisney🐭 / ショ...,1106,27,0


In [ ]:
# ミス実践
usernames_jissen = ['missjissen2021_01', 'missjissen2021_02', 'missjissen2021_03', 'missjissen2021_04']
df_jissen = get_user_info(usernames_jissen)
df_jissen['sex'] = 0
df = pd.concat([data_mrms, df_jissen], ignore_index=True)
df_jissen

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,田子森春奈 Haruna Tagomori,missjissen2021_01,17841448324499431,4586,1999.05.22\n165cm \ncontact→dm💌,1535,108,0
1,小櫃愛紗 Arisa Obitsu,missjissen2021_02,17841448406730355,4834,▫️MissJissen2021 GP \n▫️2001.12.19\n▫️お仕事・ご依頼は...,1330,34,0
2,Suzuna Fujita | 藤田涼菜,missjissen2021_03,17841448257721415,5313,Miss Jissen Contest 2021 準GP〰️\n'00 🇯🇵\nお仕事・ご依...,1497,56,0
3,𝐾𝑎𝑒𝑑𝑒,missjissen2021_04,17841448500651754,4265,Miss Jissen2021 finalist✴︎\n大学３年生(21)\n毎日の投票、沢...,1715,95,0


In [ ]:
# 成蹊大学
usernames_seikei = ['missseikei21_01', 'missseikei21_05']
df_seikei = get_user_info(usernames_seikei)
df_seikei['sex'] = 0
df = pd.concat([data_mrms, df_seikei], ignore_index=True)
df_seikei

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,石川英利佳,missseikei21_01,17841448137606193,2217,miss seikei 2021\nイシカワエリカ（19）,1159,113,0
1,Uru,missseikei21_05,17841448769112585,1837,🤍,262,44,0


In [ ]:
# ミス日大
# ミスターは上のミスターオブミスターでやっている
usernames_nucc = ['nucc21_miss02', 'seshichan_dayo']
df_nucc = get_user_info(usernames_nucc)
df_nucc['sex'] = 0
df = pd.concat([df, df_nucc], ignore_index=True)
df_nucc

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,山田瑞起 Mizuki Yamada,nucc21_miss02,17841448256885484,3320,日本大学商学部 Miss2021 GP👑\n💌サロモ撮影依頼はDMまでお願いします！\n※...,464,70,0
1,瀬下莉子,seshichan_dayo,17841449080401625,10116,せしたりこ / せしちゃん \n日本大学文理学部CHSコンテスト 準GP🦑💥\nSNS続...,1631,109,0


In [ ]:
# ミスター専修
# ミスは上のミスオブミスでやっている
usernames_senshu = ['senshu21_mr1', 'senshu21_mr2']
df_senshu = get_user_info(usernames_senshu)
df_senshu['sex'] = 1
df = pd.concat([data_mrms, df_senshu], ignore_index=True)
df_senshu

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,池田 吏久斗,senshu21_mr1,17841448668478893,1650,Mr&Miss Senshu contest 2021 GP🏆\nお仕事のご依頼等はDMまで...,990,45,1
1,髙井雄乃介(たかいゆうのすけ),senshu21_mr2,17841448535913609,976,Mr.Senshu 2021 準GP\n 経済 3年\n・2000.4.5\n・アナウンサー...,682,24,1


In [ ]:
# ミスター龍谷
# ミスは上のミスオブミスでやっている
usernames_rf = ['rf99_mr04']
df_rf = get_user_info(usernames_rf)
df_rf['sex'] = 1
df = pd.concat([data_mrms, df_rf], ignore_index=True)
df_rf

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,西川洸,rf99_mr04,17841448723220127,460,Mr.ryukoku 2021 finalist\ncafe.アイドル.バイク色々好き,24,43,1


In [ ]:
# ミスター東洋大
# ミスは上のミスオブミスでやっている
usernames_toyo = ['toyo.tux21_01']
df_toyo = get_user_info(usernames_toyo)
df_toyo['sex'] = 1
df = pd.concat([data_mrms, df_toyo], ignore_index=True)
df_toyo

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,東洋大学 タキシード No.1 清野大地,toyo.tux21_01,17841448292788906,1255,清野大地(せいのだいち)\nRoad Bike🚲/cook🍳\n東洋キャンパスコンテスト、沢...,1248,81,1


In [ ]:
# ミスSFC
usernames_sfc = ['ririko_yamada']
df_sfc = get_user_info(usernames_sfc)
df_sfc['sex'] = 0
df_sfc['reel'] = 1
df_sfc['movie_not_reel'] = 1
df = pd.concat([data_mrms, df_sfc], ignore_index=True).drop('id', axis=1)
df_sfc

,name,username,id,followers_count,biography,follows_count,media_count,sex,reel,movie_not_reel
0,山田 璃々子 | Ririko Yamada,ririko_yamada,17841448413324593,10483,Miss Keio SFC Contest 2021 グランプリ\n🍵: @ririko_o...,2565,113,0,1,1


In [ ]:
# ミスター立教
usernames_mr_rikkyo = ['rikkyo21_mr1', 'shuta_tajima', 'daikisuzuki0', 'rikkyo21_mr4', 'fuga_96story']
df_mr_rikkyo = get_user_info(usernames_mr_rikkyo)
df_mr_rikkyo['sex'] = 1
df_mr_rikkyo

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,青木優太,rikkyo21_mr1,17841447883953622,1478,2000.5.2\nMiss Mr Rikkyo 2021⚜️,1167,102,1
1,田嶋 柊太,shuta_tajima,17841448056750872,1475,東京留年大学生(23)🇯🇵\nmie suzuka\ngb250clubmania🏍\nMi...,1227,114,1
2,鈴木大輝,daikisuzuki0,17841448388419216,3092,Thank you for your support.\nShut up and love ...,3079,44,1
3,杉山龍生 sugiyama ryusei,rikkyo21_mr4,17841448287592965,1635,2000.5.9\n🌟ミスター立教2021 準GP🌟\n🏫青春高校3年C組🏫\n🐲👉 @ry...,1113,81,1
4,Fuga Toyoshima,fuga_96story,17841448295602526,1472,りっきょうううう,1275,48,1


In [ ]:
# ミス立教
usernames_ms_rikkyo = ['rikkyo21_miss1', 'nanako_hisamura', '_seera.kawabata_', 'rikkyo21_miss4', '_eri.1128.maru_']
df_ms_rikkyo = get_user_info(usernames_ms_rikkyo)
df_ms_rikkyo['sex'] = 0
df_ms_rikkyo

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,鈴木泉帆 Mizuho Suzuki,rikkyo21_miss1,17841448362046655,17774,Miss Rikkyo Contest 2021\n応援ありがとうございました🐰🍡\nTok...,1446,95,0
1,久村奈々子 Nanako Hisamura,nanako_hisamura,17841448436713420,22052,2000.11.19 (21)\nMiss Rikkyo Contest 2021 GP\n...,895,139,0
2,川畑世英良 / Seera Kawabata,_seera.kawabata_,17841448397595176,11421,"01""(20) Kagoshima → Tokyo \nMiss Mr Rikkyo Co...",895,226,0
3,田神沙羅/Sara Tagami,rikkyo21_miss4,17841448331538087,4195,Miss Rikkyo Contest 2021\n2000.09.22(21)\nKago...,459,108,0
4,仲丸恵理 Eri Nanamaru,_eri.1128.maru_,17841447980430624,8383,Miss Rikkyo Contest 2021\n応援ありがとうございました♡\n160c...,548,69,0


In [ ]:
# ミスター明学
usernames_mr_meigaku = ['21mgumr_no1', '21mgumr_no3', '21mgumr_no4', '21mgumr_no5']
df_mr_meigaku = get_user_info(usernames_mr_meigaku)
df_mr_meigaku['sex'] = 1
df_mr_meigaku

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,清水芳基-Yoshiki Shikizu,21mgumr_no4,17841448730718799,963,ミスター明治学院2021 No.4 清水芳基 (しみずよしき)\n1999.7.9 (22)...,930,54,1
1,矢留悠誠 / Yusei Yadome,21mgumr_no5,17841448806526294,2017,👑Mr. Meijigakuin Contest 2021 GP\n・\n◼︎183cm ...,1590,61,1


In [ ]:
# FRESH CAMPUS CONTEST 2021
usernames_frecam = ['happy.smile4u', '12__sar', 'kanolie_may_', 'enomoto_yuina', 'yuna.pi_chan', 'nb.grm___', 'fee_lune_', 'reika.ariga', 'frecam2021_sumika', 'matsumoto.nako', 'frecam2021_0152', 'chuchu_mmn907', 'yuna_yabana', 'kokoro_shimotsu', 'frecam2021_0401', 'ruchan___27', '_shio.ri306', 'fujishima_07', 'rira___h', '_rangel.4']
df_frecam = get_user_info(usernames_frecam)
df_frecam['sex'] = 0
df_frecam

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,北川満萌,happy.smile4u,17841402159335622,988,2003.02.04(18)\nFRESH CAMPUS CONTEST2021ファイナリス...,211,252,0
1,野口 華音,kanolie_may_,17841403777183481,1424,sophia univ. \njesty8th 👯‍♀️\nFRESH CAMPUS CO...,740,9,0
2,榎本ゆいな,enomoto_yuina,17841405366105300,6324,2002.6.16 (19) 学習院大学1年生/福岡→東京/ジャストプロ所属\n【FRESH...,127,19,0
3,馬場 菜緒 @フレキャン2021,nb.grm___,17841407436259605,1314,(19) ｜hosei.univ ｜164cm｜\n▫️フレキャン'21 ファイナリスト 1...,112,62,0
4,鄭 悠亜 (てい ゆうあ),fee_lune_,17841408841766885,1328,【ファイナル審査 10/28〜11/7】\n🌙2002/4/12\n💎東大 文科一類 \n👒...,62,43,0
5,有賀 怜香 @フレキャン2021,reika.ariga,17841421413841535,4233,(19)Meiji univ.\n〖FRESH CAMPUS CONTEST 2021 グラ...,56,55,0
6,日向すみか(ひなたすみか),frecam2021_sumika,17841446169447983,781,ひなひなです🐣\n🌼大1フレッシュキャンパスコンテストファイナリスト\n👩‍🎓Chuo un...,201,31,0
7,なーたん！松本奈子,matsumoto.nako,17841446698399371,1806,（１９）学芸 ‣‣ 関西学院大学 / 社\n【FRESH CAMPUS CONTEST 20...,58,20,0
8,堺 理沙(さかいりさ)フレキャン2021,frecam2021_0152,17841447392632675,1652,💎フレッシュキャンパスコンテスト2021 💎\n🏆審査員特別賞\n・白百合女子大学1年\n・...,96,98,0
9,矢花優奈（やばなゆうな）,yuna_yabana,17841447560810209,1256,keio univ\n❁✿✾ FRESH CAMPUS CONTEST 2021 ✾✿❁︎\...,132,48,0


In [ ]:
# FRESH CIRCLE CONTEST 2021
usernames_circle = ['manaha_ha_', 'haruka7700', 'cq_arisa_y', 'kurisaki_yuna', 'circle2021_0107', 'mikitie_0623', 'riko__kawakami', 'kaori_furumoto1113', 's.0606.s', 'circle2021_0682', 'tulip.t_t', 'maimi12122021']
df_circle = get_user_info(usernames_circle)
df_circle['sex'] = 0
df_circle

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,田辺真南葉/Manaha Tanabe,manaha_ha_,17841400486539872,13517,MISS CIRCLE CONTEST2021 審査員特別賞\n165cm 00/12/20...,169,93,0
1,ミスサー2021👑栗咲ゆうな🐹💛(ゆなぴ),kurisaki_yuna,17841436546379481,1268,◻️愛知/栗咲ゆうな🐹💛(ゆなぴ)\n◻️ミスサークルコンテスト2021 ファイナリスト👑\...,583,78,0
2,乙川えり/ Eri Otokawa,circle2021_0107,17841440740596893,1516,Miss circle contest 2021 ファイナリスト\n1999.08.12\n...,989,58,0
3,荒木 美輝 Miki Araki,mikitie_0623,17841447247575334,1664,＊ミスサークルコンテスト2021ファイナリスト Bellcida賞 受賞\n＊慶應義塾大学3...,372,101,0
4,川上 理子 (かわかみ りこ),riko__kawakami,17841447090183749,2341,🤍Miss circle contest 2021 finalist リゼクリニック賞\n...,127,42,0
5,【⭕️🥢】大橋可奈@ミスサークル2021,circle2021_0682,17841448001122929,488,【 MISS CIRCLE CONTEST 2 0 2 1 💍 C ブ ロ ッ ク 】 ⚖️...,406,14,0
6,鈴木みゆう #ミスサークル2021,tulip.t_t,17841448346779824,1142,Chiba Univ. (20)\nミスサークルコンテスト2021準グランプリ✨\nお仕事の...,124,86,0
7,大森舞美,maimi12122021,17841450385311528,248,✴︎2021ミスサークルファイナリスト✴︎\n✴︎KaJa賞受賞✴︎\nこれからの活動→超十...,70,18,0


In [ ]:
# 長崎大学 キュート部門とクール部門
usernames_nu = ['riixho', 'yu__ppy__', 'nu_contest_no.3', '_nozomitomioka']
df_nu = get_user_info(usernames_nu)
#df_circle['sex'] = 0
df_nu

,name,username,id,followers_count,biography,follows_count,media_count
0,yuna,yu__ppy__,17841403050142072,1549,― '99\n― 𝐟𝐮𝐤𝐮𝐨𝐤𝐚ｰ𝐧𝐚𝐠𝐚𝐬𝐚𝐤𝐢\nー 🥞☕️\nー ご依頼はフォロー後d...,710,313
1,nOzomi tomiokA,_nozomitomioka,17841432922966887,131,ﾉｿﾞﾐﾄﾐｵｶ 20っさい\n好きなお洋服着てます\n系統は知らんけども好きなお洋服着てます,24,40


> [pandas.DataFrame.at](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html)

In [ ]:
# 日大芸術学部
usernames_nichigei = ['burasutopher', '___jazmin.arcoiris', 'urlocalmixedgirll']
df_nichigei = get_user_info(usernames_nichigei)
df_nichigei['sex'] = 0
df_nichigei.at[0, 'sex'] = 1
df_nichigei

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,christopher,burasutopher,17841449271674929,150,被写体用のアカウント\n写真を撮ってくれるという方、またお仕事等はDMまでお願いします,144,8,1
1,ミクス日芸No.3 MARIA ‐ ﾏ ﾘ ｱ ‐,___jazmin.arcoiris,17841436931295897,647,Mx日芸コンテスト2021🌟💎 DHC賞🏅\nNon-Binary ; they / the...,533,44,0
2,Rio,urlocalmixedgirll,17841401246495544,722,NUA Film \nミクス日芸コンテスト2021グランプリ👑,865,31,0


In [ ]:
# ミス医科歯科
usernames_tmdu = ['misstmdu202101', 'misstmdu202102']
df_tmdu = get_user_info(usernames_tmdu)
df_tmdu['sex'] = 0
df_tmdu

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,安田令,misstmdu202101,17841448373611240,800,一般的歯学生\nDorrydoll賞、準グランプリありがとうございました💐\n被写体依頼、そ...,270,59,0


In [ ]:
# ミス熊本大
usernames_kumadai = ['miss_kumadai_no.5']
df_kumadai = get_user_info(usernames_kumadai)
df_kumadai['sex'] = 0
df_kumadai

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,ミス熊大No.5 安川未蘭,miss_kumadai_no.5,17841448869144063,1920,第十回ミス熊大コンテストグランプリ👑🤍\n応援本当にありがとうございました😭✨,1449,5,0


In [ ]:
# ミス横浜市立大
usernames_msycu = ['airi_sugimoto92']
df_msycu = get_user_info(usernames_msycu)
df_msycu['sex'] = 0
df_msycu

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,杉本藍梨,airi_sugimoto92,17841448668387884,1651,グランプリをいただくことができました👑\n応援ありがとうございました❣️\nDMのお返事はで...,1838,26,0


In [ ]:
# ミスター東京大
usernames_mr_todai = ['mrtodai21_01', 'mrtodai21_02', 'mrtodai21_03', 'mrtodai21_04', 'mrtodai21_05']
df_mr_todai = get_user_info(usernames_mr_todai)
df_mr_todai['sex'] = 1
df_mr_todai

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,ミスター東大No.1 いーだ,mrtodai21_01,17841448138411787,889,ミス&ミスター東大コンテスト2021 No.1 いーだ\n文科2類 1年 河合塾本郷校出身の...,48,4,1
1,小田原光一,mrtodai21_02,17841448504015091,492,ミス&ミスター東大コンテスト2021\nNo.2,69,21,1
2,材木太一,mrtodai21_03,17841448656679654,2473,ミスター東大2021 グランプリ👑,2293,36,1
3,ミスター東大No.4 山村裕次郎,mrtodai21_04,17841448433007904,1987,◆ミスター東大2021 No.4\n◆神戸出身 灘高ラグビー部/東大理三 /医学部4年\n◆...,1326,70,1
4,Mr東大 No.5 新郷史弥,mrtodai21_05,17841448668109164,1233,応援本当にありがとうございました。,1081,36,1


In [ ]:
# ミス東大
usernames_ms_todai = ['misstodai21_01', 'misstodai21_02', 'sally_._yllas', 'misstodai21_04', 'misstodai21_05']
df_ms_todai = get_user_info(usernames_ms_todai)
df_ms_todai['sex'] = 0
df_ms_todai

,name,username,id,followers_count,biography,follows_count,media_count,sex
0,茅野 愛理 * Airi Chino,misstodai21_01,17841448708277085,1464,ミス東大2021ファイナリスト\nLike: SKE48🧡パン🍞サラダ🥗ワイン🍷アイス🍨ギョ...,1049,44,0
1,長谷川遥花 Haruka Hasegawa,misstodai21_02,17841448724356405,9043,沢山の応援ありがとうございました💗\n◯2002.5.5 (19)\n◯ミス東大2021準グ...,2027,117,0
2,千葉彩里衣/Sally Chiba,sally_._yllas,17841448639819612,2060,‘01(19)\n𝘊𝘰𝘯𝘵𝘢𝘤𝘵→𝘥𝘮💌\nミス東大2021応援ありがとうございました❕,431,80,0
3,ミス東大No.4 上野万里,misstodai21_04,17841448483105255,3189,東京大学文科二類2年\nミス東大2021グランプリ\n福岡出身\n2000.05.02,218,34,0
4,河村 汐莉,misstodai21_05,17841448527833383,861,＊かわむら しおり\n＊ミス東大コンテスト2021\n＊新領域創成科学研究科 修士1年\n＊...,812,22,0


In [ ]:
df = pd.concat([data_mrms, df_ms_todai], ignore_index=True).drop('id', axis=1)

In [ ]:
print(len(data_mrms['username'].value_counts()) == len(data_mrms))  # usernameの被りがないか確認
print(len(df['username'].value_counts()) == len(df))  # usernameの被りがないか確認

True
True


In [ ]:
data_mrms = df

In [ ]:
data_mrms.to_csv('/content/drive/My Drive/data_mrms.csv')